In [ ]:
from PyQt6.QtWidgets import (
    QApplication, QDialog, QComboBox, QFormLayout, QLineEdit,
    QVBoxLayout, QPushButton, QFileDialog, QDialogButtonBox
)
#for AGOL 
from arcgis.gis import GIS
from arcgis.features import GeoAccessor, GeoSeriesAccessor, FeatureLayerCollection
from arcgis.map import Map
from IPython.display import display

#for geoprocessing 
from arcgis.geoprocessing import import_toolbox
import arcpy
from arcpy.lr import *

#for file/folder manipulation
from pathlib import Path
import zipfile
from zipfile import ZipFile
import os
import glob
import shutil 
from datetime import datetime
import time

#for downloading google sheets
import webbrowser


class Window(QDialog):
    def __init__(self):
        super().__init__(parent=None)
        self.setWindowTitle("Linear Referencing/ Dynamic Segmentation")

        dialogLayout = QVBoxLayout()
        formLayout = QFormLayout()

        # User input fields
        self.usernameField = QLineEdit()
        self.passwordField = QLineEdit()
        self.passwordField.setEchoMode(QLineEdit.EchoMode.Password)  # Hide password
        self.sheetURLField = QLineEdit()

        formLayout.addRow("AGOL Username:", self.usernameField)
        formLayout.addRow("AGOL Password:", self.passwordField)
        formLayout.addRow("Google Sheet URL:", self.sheetURLField)

        # File selection
        self.fileButton = QPushButton("Select File Location")
        self.fileButton.clicked.connect(self.open_file_dialog)
        self.filePath = QLineEdit()
        self.filePath.setReadOnly(True)

        formLayout.addRow("File Location:", self.fileButton)
        #formLayout.addRow("Selected File:", self.filePath)

        # Layer details
        self.layerIdField = QLineEdit()
        self.layerUpdateField = QLineEdit()
        formLayout.addRow("Layer Id:", self.layerIdField)
        formLayout.addRow("What Layer would you like to update?", self.layerUpdateField)

        # Points or Lines selection
        self.comboBox = QComboBox()
        self.comboBox.addItems(["Points", "Lines"])
        formLayout.addRow("Points or Lines:", self.comboBox)

        dialogLayout.addLayout(formLayout)

        # Buttons
        self.buttons = QDialogButtonBox()
        self.runButton = QPushButton("Run")
        self.cancelButton = QPushButton("Cancel")

        self.runButton.clicked.connect(self.run_script)
        self.cancelButton.clicked.connect(self.reject)

        self.buttons.addButton(self.runButton, QDialogButtonBox.ButtonRole.AcceptRole)
        self.buttons.addButton(self.cancelButton, QDialogButtonBox.ButtonRole.RejectRole)

        dialogLayout.addWidget(self.buttons)
        self.setLayout(dialogLayout)

    def open_file_dialog(self):
        """Opens file explorer for selecting a directory."""
        folder = QFileDialog.getExistingDirectory(self, "Select Directory")
        if folder:
            self.filePath.setText(folder)

    def run_script(self):
        """Passes user inputs to the script and executes it."""
        username = self.usernameField.text()
        password = self.passwordField.text()
        sheet_url = self.sheetURLField.text()
        file_location = self.filePath.text()
        layer_id = self.layerIdField.text()
        update_layer = self.layerUpdateField.text()
        points_or_lines = self.comboBox.currentText()

        if not all([username, password, sheet_url, file_location, layer_id, update_layer]):
            print("Please fill in all required fields.")
            return

        execute_script(username, password, sheet_url, file_location, layer_id, update_layer, points_or_lines)
        self.accept()


def execute_script(username, password, sheet_url, file_location, layer_id, update_layer, points_or_lines):
    """Executes the user's script with the provided inputs."""

    # Connect to AGOL
    gis = GIS("https://cdot.maps.arcgis.com/home/index.html", username, password)

    # Create a timestamped directory
    current_date = datetime.now().strftime("_%Y-%m-%d.%H.%M")
    data_folder = os.path.join(file_location, 'Data' + current_date)
    os.makedirs(data_folder, exist_ok=True)

    # Pull Route Layer
    routes_id = 'cac39bef197b46be8a6e8efa46af61f3'
    routes_dl = gis.content.get(routes_id)

    # Download Routes as ZIP
    result = routes_dl.export(routes_dl.title, 'Shapefile')
    zip_path = os.path.join(data_folder, 'Routes.zip')
    result.download(save_path=data_folder)
    result.delete()  # Clean up AGOL

    # Extract ZIP
    routes_data = os.path.join(data_folder, 'Routes')
    os.makedirs(routes_data, exist_ok=True)
    with ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(routes_data)

    # Download CSV from Google Sheets
    url_1 = sheet_url.replace("/edit?gid=0#gid=0", "/export?format=csv")
    webbrowser.open(url_1)

    # Wait and find the latest CSV from Chrome downloads
    time.sleep(10)
    chrome_folder = os.path.join(os.path.expanduser("~"), "Downloads\Chrome", "*.csv")
    csv_dl = max(glob.glob(chrome_folder), key=os.path.getctime)
    shutil.move(csv_dl, data_folder)

    # Set variables for route event layer
    routes = os.path.join(routes_data, 'Routes.shp')
    rid = "ROUTE"
    tbl = os.path.join(data_folder, os.path.basename(csv_dl))
    props = "ROUTE POINT BeginRefPoint EndRefPoint" if points_or_lines == "Points" else "ROUTE LINE BeginRefPoint EndRefPoint"
    lyr = update_layer

    # Make Route Event Layer
    arcpy.lr.MakeRouteEventLayer(routes, rid, tbl, props, lyr)

    # Export Features
    outFC_path = os.path.join(data_folder, update_layer)
    os.makedirs(outFC_path, exist_ok=True)
    outFeatureClass = os.path.join(outFC_path, update_layer)
    arcpy.conversion.ExportFeatures(lyr, outFeatureClass)

    # Zip new shapefile
    zip_folder(outFC_path, outFC_path + ".zip", exclude_types=['.lock'])

    # Update AGOL layer
    new_lyr_collection = FeatureLayerCollection.fromitem(gis.content.get(layer_id))
    new_lyr_collection.manager.overwrite(outFC_path + ".zip")


def zip_folder(folder_path, output_path, exclude_types):
    """Zips the specified folder while excluding certain file types."""
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                if not any(file.endswith(ext) for ext in exclude_types):
                    zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), folder_path))


if __name__ == "__main__":
    app = QApplication([])
    window = Window()
    if window.exec():
        print("Layer successfully Updated.")
    sys.exit(app.exec())

C:\Users\wuethrichca\AppData\Local\Temp\ipykernel_20372\1730444981.py:32: DeprecationWarning: sipPyTypeDict() is deprecated, the extension module should use sipPyTypeDictRef() instead
  super().__init__(parent=None)


Layer successfully Updated.
